In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# смотреть под https://coub.com/view/10kyav
# велосипед написан через global, не предел мечтаний - но работает.
# значения ключа exchange полученны влоб с SQL сервера, в оригинале там 200К вариантов query строки.
# сплит и ulr-parser показали себя плохо, поэтому был изобретён велосипед- парсер через .str.contains()
# велосипед бьёт выборку на объекты и возвращает pd.DataFrame в каждом отдельном случае, всего 4000+ проход без ошибок.
# жрёт оперативу, цена вопроса х2 по стоимости от чтения оригинального датафрейма. попытки отказа от global пока что фейл.

In [2]:
df_1 = pd.read_csv('J:/Data_mks/sub_rebills_2017-08-01_2017-08-31_f/sub_rebills_2017-08-01_2017-08-31_f.csv', sep='\t', encoding='utf-8')

In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000730 entries, 0 to 5000729
Data columns (total 21 columns):
sub_uuid             object
service_id           int64
URL                  object
Подключение          object
Юр.лицо              object
ОСС                  object
date_init            object
date_activation      object
date_stop            object
msisdn               int64
referrer             object
query                object
ip                   object
ua                   object
error_code           object
is_active            object
rebill_uuid          object
rebill_date          object
rebill_error_code    float64
rebill_status        float64
ua_black_list        int64
dtypes: float64(2), int64(3), object(16)
memory usage: 801.2+ MB


In [4]:
df_1.head()

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
0,3c9710c0-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:09,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.4.140,Mozilla/5.0 (Linux; Android 6.0; U20 Build/MRA...,0,0,NaN,NaN,NaN,NaN,0
1,3dd8f1b0-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0
2,3e7bb936-7633-11e7-b3ea-728ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:12,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=epom&f...,213.87.163.0,Mozilla/5.0 (Linux; Android 6.0; MZ-MEIZU_M5 B...,0,0,NaN,NaN,NaN,NaN,1
3,44c4eeb6-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:22,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=epom&...,213.87.152.167,Mozilla/5.0 (Linux; Android 5.0.1; Lenovo TAB ...,0,0,NaN,NaN,NaN,NaN,0
4,45530142-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:23,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=epom&f...,213.87.128.9,Mozilla/5.0 (Linux; Android 4.3; GT-I9500 Buil...,0,0,NaN,NaN,NaN,NaN,0


In [5]:
# эта функция размечает выборку по рекламным сетям.
def select():
    global Bexc 
    Bexc = df_1[df_1['query'].str.contains('exchange=%7Bexc%7D', na=False)]
    global BEXCHANGE_ID 
    BEXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=BEXCHANGE_ID', na=False)]
    global EXCHANGE_ID 
    EXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=EXCHANGE_ID', na=False)]
    global axonix 
    axonix = df_1[df_1['query'].str.contains('exchange=axonix', na=False)]
    global smartyads 
    smartyads = df_1[df_1['query'].str.contains('exchange=smartyads', na=False)]
    global undef 
    undef = df_1[df_1['query'].str.contains('exchange=undef', na=False)]
    global arteebee
    arteebee = df_1[df_1['query'].str.contains('exchange=arteebee', na=False)]
    global between 
    between = df_1[df_1['query'].str.contains('exchange=between', na=False)]
    global clickky 
    clickky = df_1[df_1['query'].str.contains('exchange=clickky', na=False)]
    global google 
    google = df_1[df_1['query'].str.contains('exchange=google', na=False)]
    global gothamads 
    gothamads = df_1[df_1['query'].str.contains('exchange=gothamads', na=False)]
    global mobilda 
    mobilda = df_1[df_1['query'].str.contains('exchange=mobilda', na=False)]
    global repeater 
    repeater = df_1[df_1['query'].str.contains('exchange=repeater', na=False)]
    global republer 
    republer = df_1[df_1['query'].str.contains('exchange=republer', na=False)]
    global sape 
    sape = df_1[df_1['query'].str.contains('exchange=sape', na=False)]  
    global clickky2 
    clickky2 = df_1[df_1['query'].str.contains('exchange=clickky2', na=False)]
    global exoclick
    exoclick = df_1[df_1['query'].str.contains('exchange=exoclick', na=False)]
    global popcash 
    popcash = df_1[df_1['query'].str.contains('exchange=popcash', na=False)]
    global popunder 
    popunder = df_1[df_1['query'].str.contains('exchange=popunder', na=False)]
    global propeller 
    propeller = df_1[df_1['query'].str.contains('exchange=propeller', na=False)]
    global yabbi 
    yabbi = df_1[df_1['query'].str.contains('exchange=yabbi', na=False)]
    global activerevenue
    activerevenue = df_1[df_1['query'].str.contains('exchange=activerevenue', na=False)]
    global fallback 
    fallback = df_1[df_1['query'].str.contains('exchange=fallback', na=False)]
    global innex 
    innex = df_1[df_1['query'].str.contains('exchange=innex', na=False)]
    global juicy 
    juicy = df_1[df_1['query'].str.contains('exchange=juicy', na=False)]
    global airpush 
    airpush = df_1[df_1['query'].str.contains('exchange=airpush', na=False)]
    global leadbolt 
    leadbolt = df_1[df_1['query'].str.contains('exchange=leadbolt', na=False)]
    global something
    something = df_1[df_1['query'].str.contains('exchange=something', na=False)]
    global hilltop 
    hilltop = df_1[df_1['query'].str.contains('exchange=hilltop', na=False)]
    global propellerads 
    propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]
    global mobicow 
    mobicow = df_1[df_1['query'].str.contains('exchange=mobicow', na=False)]
    global openx 
    openx = df_1[df_1['query'].str.contains('exchange=openx', na=False)]
    global smaato 
    smaato = df_1[df_1['query'].str.contains('exchange=smaato', na=False)]
    global tonic 
    tonic = df_1[df_1['query'].str.contains('exchange=tonic', na=False)]
    global traffichunt 
    traffichunt = df_1[df_1['query'].str.contains('exchange=traffichunt', na=False)]
    global zeropark 
    zeropark = df_1[df_1['query'].str.contains('exchange=zeropark', na=False)]
    global admobile
    admobile = df_1[df_1['query'].str.contains('exchange=admobile', na=False)]
    global tapxx 
    tapxx = df_1[df_1['query'].str.contains('exchange=tapxx', na=False)]
    global teasernet 
    teasernet = df_1[df_1['query'].str.contains('exchange=teasernet', na=False)]
    global visitweb
    visitweb = df_1[df_1['query'].str.contains('exchange=visitweb', na=False)]
    global epom
    epom = df_1[df_1['query'].str.contains('exchange=epom', na=False)]
    global plugrush
    plugrush = df_1[df_1['query'].str.contains('exchange=plugrush', na=False)]
    global kadam
    kadam = df_1[df_1['query'].str.contains('exchange=kadam', na=False)]
    global minimob
    minimob = df_1[df_1['query'].str.contains('exchange=minimob', na=False)]
    global peakadx
    peakadx = df_1[df_1['query'].str.contains('exchange=peakadx', na=False)]
    global a_propellerads
    a_propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]

In [6]:
select()

In [8]:
type(axonix)

pandas.core.frame.DataFrame

In [12]:
axonix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1263244 entries, 0 to 5000714
Data columns (total 21 columns):
sub_uuid             1263244 non-null object
service_id           1263244 non-null int64
URL                  1263244 non-null object
Подключение          1263244 non-null object
Юр.лицо              1263244 non-null object
ОСС                  1263244 non-null object
date_init            1263244 non-null object
date_activation      44532 non-null object
date_stop            31403 non-null object
msisdn               1263244 non-null int64
referrer             200044 non-null object
query                1263244 non-null object
ip                   1263244 non-null object
ua                   1263244 non-null object
error_code           1263244 non-null object
is_active            1263244 non-null object
rebill_uuid          44498 non-null object
rebill_date          44498 non-null object
rebill_error_code    44498 non-null float64
rebill_status        44498 non-null float64


In [9]:
# эта функция фильтрует и показывает результат, но не возвращает объекты в стек.
def exchange():
    Bexc = df_1[df_1['query'].str.contains('exchange=%7Bexc%7D', na=False)]
    BEXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=BEXCHANGE_ID', na=False)]
    EXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=EXCHANGE_ID', na=False)]
    axonix = df_1[df_1['query'].str.contains('exchange=axonix', na=False)]
    smartyads = df_1[df_1['query'].str.contains('exchange=smartyads', na=False)]
    undef = df_1[df_1['query'].str.contains('exchange=undef', na=False)]
    arteebee = df_1[df_1['query'].str.contains('exchange=arteebee', na=False)]
    between = df_1[df_1['query'].str.contains('exchange=between', na=False)]
    clickky = df_1[df_1['query'].str.contains('exchange=clickky', na=False)]
    google = df_1[df_1['query'].str.contains('exchange=google', na=False)]
    gothamads = df_1[df_1['query'].str.contains('exchange=gothamads', na=False)]
    mobilda = df_1[df_1['query'].str.contains('exchange=mobilda', na=False)]
    repeater = df_1[df_1['query'].str.contains('exchange=repeater', na=False)]
    republer = df_1[df_1['query'].str.contains('exchange=republer', na=False)]
    sape = df_1[df_1['query'].str.contains('exchange=sape', na=False)]    
    clickky2 = df_1[df_1['query'].str.contains('exchange=clickky2', na=False)]
    exoclick = df_1[df_1['query'].str.contains('exchange=exoclick', na=False)]
    popcash = df_1[df_1['query'].str.contains('exchange=popcash', na=False)]
    popunder = df_1[df_1['query'].str.contains('exchange=popunder', na=False)]
    propeller = df_1[df_1['query'].str.contains('exchange=propeller', na=False)]
    yabbi = df_1[df_1['query'].str.contains('exchange=yabbi', na=False)]
    activerevenue = df_1[df_1['query'].str.contains('exchange=activerevenue', na=False)]
    fallback = df_1[df_1['query'].str.contains('exchange=fallback', na=False)]
    innex = df_1[df_1['query'].str.contains('exchange=innex', na=False)]
    juicy = df_1[df_1['query'].str.contains('exchange=juicy', na=False)]
    airpush = df_1[df_1['query'].str.contains('exchange=airpush', na=False)]
    leadbolt = df_1[df_1['query'].str.contains('exchange=leadbolt', na=False)]
    something = df_1[df_1['query'].str.contains('exchange=something', na=False)]
    hilltop = df_1[df_1['query'].str.contains('exchange=hilltop', na=False)]
    propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]
    mobicow = df_1[df_1['query'].str.contains('exchange=mobicow', na=False)]
    openx = df_1[df_1['query'].str.contains('exchange=openx', na=False)]
    smaato = df_1[df_1['query'].str.contains('exchange=smaato', na=False)]
    tonic = df_1[df_1['query'].str.contains('exchange=tonic', na=False)]
    traffichunt = df_1[df_1['query'].str.contains('exchange=traffichunt', na=False)]
    zeropark = df_1[df_1['query'].str.contains('exchange=zeropark', na=False)]
    admobile = df_1[df_1['query'].str.contains('exchange=admobile', na=False)]
    tapxx = df_1[df_1['query'].str.contains('exchange=tapxx', na=False)]
    teasernet = df_1[df_1['query'].str.contains('exchange=teasernet', na=False)]
    visitweb = df_1[df_1['query'].str.contains('exchange=visitweb', na=False)]
    epom = df_1[df_1['query'].str.contains('exchange=epom', na=False)]
    plugrush = df_1[df_1['query'].str.contains('exchange=plugrush', na=False)]
    kadam = df_1[df_1['query'].str.contains('exchange=kadam', na=False)]
    minimob = df_1[df_1['query'].str.contains('exchange=minimob', na=False)]
    peakadx = df_1[df_1['query'].str.contains('exchange=peakadx', na=False)]
    a_propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]
    print('Bexc', len(Bexc), 'axonix', len(axonix), 'smartyads', len(smartyads), 'undef', len(undef),'arteebee', len(arteebee), 
                 'between', len(between), 'clickky', len(clickky), 'google', len(google), 'gothamads',len(gothamads), 'mobilda',len(mobilda),
                 'repeater',len(repeater), 'republer', len(republer), 'sape', len(sape), 'clickky2', len(clickky2),'exoclick',len(exoclick),
                 'popcash',len(popcash), 'popunder',len(popunder), 'propeller',len(propeller), 'yabbi',len(yabbi), 'activerevenue', len(activerevenue),
                'fallback', len(fallback), 'innex', len(innex), 'juicy',len(juicy), 'airpush', len(airpush), 'leadbolt', len(leadbolt),
                 'something', len(something), 'hilltop',len(hilltop), 'propellerads',len(propellerads), 'mobicow', len(mobicow),'openx',len(openx),
                 'smaato',len(smaato), 'tonic',len(tonic), 'traffichunt',len(traffichunt), 'zeropark', len(zeropark), 'admobile', len(admobile),
                 'tapxx', len(tapxx), 'teasernet', len(teasernet), 'visitweb', len(visitweb), 'BEXCHANGE_ID',len(BEXCHANGE_ID),
                 'EXCHANGE_ID',len(EXCHANGE_ID) ,'epom', len(epom), 'plugrush',len(plugrush), 'kadam', len(kadam),
                  'minimob', len(minimob), 'peakadx', len(peakadx), 'a_propellerads', len(a_propellerads))

In [10]:
exchange()

Bexc 2 axonix 1263244 smartyads 36737 undef 0 arteebee 1 between 2902 clickky 1138501 google 745 gothamads 3705 mobilda 588196 repeater 1774 republer 3 sape 223286 clickky2 0 exoclick 0 popcash 0 popunder 0 propeller 0 yabbi 0 activerevenue 0 fallback 0 innex 0 juicy 0 airpush 0 leadbolt 0 something 0 hilltop 0 propellerads 0 mobicow 0 openx 0 smaato 0 tonic 0 traffichunt 0 zeropark 0 admobile 0 tapxx 0 teasernet 0 visitweb 0 BEXCHANGE_ID 0 EXCHANGE_ID 0 epom 576375 plugrush 0 kadam 0 minimob 0 peakadx 0 a_propellerads 0


In [ ]:
# stat функция пока что лучше всех по производительности, но иногда возаращет косяк в выводе от normalize.

In [13]:
# эта функция выводи статистику по одной сети.
def stat(init, subs, stops, subs_active, subs_fr_error, subs_type_error, 
         rebills, rebills_status, rebills_fr_error, rebills_type_error,
         users, users_unic, users_unic_source, users_ip, users_unic_ip):
    init = init['date_init'].value_counts().sum()
    subs = subs['date_activation'].value_counts().sum()
    stops = stops['date_stop'].value_counts().sum()
    subs_active = subs_active['is_active'].value_counts()
    subs_fr_error = subs_fr_error['error_code'].value_counts(normalize=True)
    subs_type_error = subs_type_error['error_code'].value_counts()
    rebills = rebills['rebill_date'].value_counts().sum()
    rebills_status = rebills_status['rebill_status'].value_counts()
    rebills_fr_error = rebills_fr_error['rebill_error_code'].value_counts(normalize=True)
    rebills_type_error = rebills_type_error['rebill_error_code'].value_counts()
    users = users['msisdn'].value_counts().sum()
    users_unic = users_unic['msisdn'].drop_duplicates().values.tolist()
    users_unic_source = users_unic_source['referrer'].drop_duplicates().values.tolist()
    users_ip = users_ip['ip'].values.tolist()
    users_unic_ip = users_unic_ip['ip'].drop_duplicates().values.tolist()
    return print('Число инитов', init,'Число подписок', subs, 'Число стопов', stops,'\n','Состояние подписок','\n',
                 subs_active,'\n', 
                 'Частоты ошибок по подпискам','\n',subs_fr_error,'\n', 'Типы ошибок по подпискам','\n', subs_type_error, '\n',
                 'Число ребиллов', rebills, '\n','Успешность ребильности', '\n',rebills_status, '\n',
                 'Частоты ошибок по ребиллам', '\n',rebills_fr_error, '\n','Типы ошибок по ребиллам', '\n',rebills_type_error,'Число юзеров', users,'\n',
                 'Число уникальных юезров', len(users_unic),'\n', 'Число уникальных источников', len(users_unic_source),'\n',
                 'Число ip адресов', len(users_ip),'\n', 'Число уникальных ip адресов',len(users_unic_ip))

In [14]:
stat(axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix,axonix)

Число инитов 1263244 Число подписок 44532 Число стопов 31403 
 Состояние подписок 
 0    1244649
1      13162
0       5433
Name: is_active, dtype: int64 
 Частоты ошибок по подпискам 
 0      8.988010e-01
8      6.097555e-02
14     9.809665e-03
103    7.893170e-03
106    6.305987e-03
105    3.829031e-03
110    3.717413e-03
0      3.585214e-03
107    2.666152e-03
9      7.821134e-04
8      6.024173e-04
6      4.179715e-04
101    1.891954e-04
16     1.741548e-04
113    1.124090e-04
103    9.341030e-05
112    2.295677e-05
14     1.662387e-05
10     2.374838e-06
110    1.583225e-06
6      7.916127e-07
16     7.916127e-07
Name: error_code, dtype: float64 
 Типы ошибок по подпискам 
 0      1135405
8        77027
14       12392
103       9971
106       7966
105       4837
110       4696
0         4529
107       3368
9          988
8          761
6          528
101        239
16         220
113        142
103        118
112         29
14          21
10           3
110          2
6            

In [15]:
# эта штука подавляет вывод косяков.
axonix['error_code'].value_counts(normalize=True).apply(lambda x: format(x, 'f'))

0      0.898801
8      0.060976
14     0.009810
103    0.007893
106    0.006306
105    0.003829
110    0.003717
0      0.003585
107    0.002666
9      0.000782
8      0.000602
6      0.000418
101    0.000189
16     0.000174
113    0.000112
103    0.000093
112    0.000023
14     0.000017
10     0.000002
110    0.000002
6      0.000001
16     0.000001
Name: error_code, dtype: object

In [ ]:
# неспешно переписываю велосипед в рамках отказа от global, вариантов как сделать красиво много - рабочих неприлично мало.
# .str.contains() божественнен и решает половину проблем датасаенса - с ним можно делать так https://coub.com/view/exeep
# однако, мы не ищем лёгких путей, поэтому на выходе будет так https://coub.com/view/a7c7t
# пандас не медленный это выборки большие - а руки кривые, велосипед работает за 10 минут.
# вопрос - что именно можно вернуть в стэк через return от пандас стакл очень актуален.
# возможно лучший вариантом будет написание класса. не хотелось бы. в классы я не смог.